In [3]:
import pennylane as qml
from pennylane import numpy as np


In [4]:
def prep_state():
    qml.Hadamard(wires=0)


In [5]:
def encoding_map_repetition(wires):
    for wire in wires[1:]:
        qml.CNOT(wires=[0, wire])
    for wire in wires:
        qml.Hadamard(wires=wire)

In [6]:
dev3_rep = qml.device("default.qubit", wires=3)
print(qml.draw(encoding_map_repetition)(dev3_rep.wires))

0: ─╭●─╭●──H─┤  
1: ─╰X─│───H─┤  
2: ────╰X──H─┤  


In [7]:
def variational_circuit_3(alpha, wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

    qml.IsingZZ(alpha, wires=[2, 0])
    qml.IsingZZ(alpha, wires=[0, 1])
    qml.IsingZZ(alpha, wires=[1, 2])

In [8]:
# dev3_VGQEC = qml.device("default.qubit", wires=3)
def VGQEC_3_encoding(alpha, wires):
    qml.Hadamard(wires=0)  
    A = (1.0 + np.exp(1j * alpha)) / 2.0
    B = (1.0 - np.exp(1j * alpha)) / 2.0
    ancillas = np.array([A, 0, 0, B], dtype=complex)
    qml.MottonenStatePreparation(ancillas, wires=[1, 2])

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])




In [9]:
dev3 = qml.device("default.qubit", wires=3)
@qml.qnode(dev3)
def VGQEC_3(alpha, wires):
    VGQEC_3_encoding(alpha, wires)

    return qml.state()

print(VGQEC_3(np.pi/2, dev3.wires))

[0.35355339+0.35355339j 0.        +0.j         0.        +0.j
 0.35355339-0.35355339j 0.35355339-0.35355339j 0.        +0.j
 0.        +0.j         0.35355339+0.35355339j]


In [10]:
print(VGQEC_3(0, np.pi/2, dev3.wires))

TypeError: VGQEC_3() takes 2 positional arguments but 3 were given

In [11]:
dev5_rep = qml.device("default.qubit", wires=5)
print(qml.draw(encoding_map_repetition)(dev5_rep.wires))

0: ─╭●─╭●─╭●─╭●──H─┤  
1: ─╰X─│──│──│───H─┤  
2: ────╰X─│──│───H─┤  
3: ───────╰X─│───H─┤  
4: ──────────╰X──H─┤  


In [ ]:
# def encoding_map_513(wires):
#     # encoding_map_repetition(wires)
#     # qml.IsingZZ(-np.pi/2, wires=[4, 0])
#     # qml.IsingZZ(-np.pi/2, wires=[0, 1])
#     # qml.IsingZZ(-np.pi/2, wires=[1, 2])
#     # qml.IsingZZ(-np.pi/2, wires=[2, 3])
#     # qml.IsingZZ(-np.pi/2, wires=[3, 4])
#     qml.Hadamard(wires=wires[0])
#     qml.CNOT(wires=[0, 1])
#     qml.CNOT(wires=[0, 2])
#     qml.CNOT(wires=[0, 3])
#     qml.CNOT(wires=[0, 4])

#     qml.Hadamard(wires=wires[1])
#     qml.CNOT(wires=[1, 0])
#     qml.T(wires=wires[0])
#     qml.CNOT(wires=[1, 0])
#     qml.T(wires=wires[0])
#     qml.Hadamard(wires=wires[1])
#     qml.T(wires=wires[0])

#     qml.CNOT(wires=[2, 0])
#     qml.T(wires=wires[0])
#     qml.CNOT(wires=[2, 0])
#     qml.T(wires=wires[0])
#     qml.S(wires=wires[0])

#     qml.Hadamard(wires=wires[2])
#     qml.CNOT(wires=[2, 0])
#     qml.T(wires=wires[0])
#     qml.CNOT(wires=[2, 0])
#     qml.T(wires=wires[0])
#     qml.Hadamard(wires=wires[2])
#     qml.T(wires=wires[0])

#     qml.CNOT(wires=[3, 0])
#     qml.T(wires=wires[0])
#     qml.CNOT(wires=[3, 0])
#     qml.T(wires=wires[0])
#     qml.S(wires=wires[0])
    


In [34]:
dev5_513 = qml.device("default.qubit", wires=5)
@qml.qnode(dev5_513)
def circ_5():
    encoding_map_513(dev5_513.wires)
    return qml.state()

In [36]:
import pennylane as qml
from pennylane import numpy as np

def rz(pi_frac, wire):
    qml.RZ(np.pi * pi_frac, wires=wire)

def ry(pi_frac, wire):
    qml.RY(np.pi * pi_frac, wires=wire)

def encoding_map_513_general(wires):
    Q1, Q5, Q2, Q4, Q3 = wires

    # Step 1: Hadamards and S gate
    qml.Hadamard(wires=Q5)
    qml.Hadamard(wires=Q2)
    qml.Hadamard(wires=Q4)
    qml.Hadamard(wires=Q3)
    qml.S(wires=Q3)

    # Step 2: First round of ZYZ rotations
    rz(4/3, Q1); ry(1, Q1); rz(7/6, Q1)
    rz(5/6, Q5); ry(1, Q5); rz(7/6, Q5)
    rz(5/4, Q2); ry(0.5, Q2); rz(1, Q2)
    rz(0.5, Q4); ry(0.5, Q4); rz(0.25, Q4)
    rz(0.75, Q3); ry(0.5, Q3); rz(0.5, Q3)

    # Step 3: CNOT chain
    qml.CNOT(wires=[Q1, Q5])
    qml.CNOT(wires=[Q5, Q2])
    qml.CNOT(wires=[Q2, Q4])
    qml.CNOT(wires=[Q4, Q3])

    # Step 4: ZYZ round 2 (some negative Y angles)
    rz(1, Q1); ry(-0.5, Q1); rz(0.75, Q1)
    rz(1, Q5); ry(-0.5, Q5); rz(0.75, Q5)
    rz(1, Q2); ry(-0.5, Q2); rz(0.75, Q2)
    rz(0.25, Q4); ry(-0.5, Q4); rz(0.75, Q4)
    rz(0.5, Q3); ry(-0.5, Q3); rz(1.25, Q3)

    # Step 5: Second CNOT chain
    qml.CNOT(wires=[Q1, Q5])
    qml.CNOT(wires=[Q5, Q2])
    qml.CNOT(wires=[Q2, Q4])
    qml.CNOT(wires=[Q4, Q3])

    # Step 6: Final rotations
    rz(1, Q1); ry(0.5, Q1); rz(1, Q1)
    rz(0.75, Q5); ry(0.5, Q5); rz(0.75, Q5)
    rz(0.75, Q2); ry(0.5, Q2); rz(1, Q2)
    rz(0.75, Q4); ry(0.5, Q4); rz(1.5, Q4)


In [38]:
dev = qml.device("default.qubit", wires=5)

@qml.qnode(dev)
def circuit():

    encoding_map_513_general(wires=[0, 1, 2, 3, 4])
    return qml.state()


In [40]:
state = circuit()
print(state)

[-0.02783789-0.06051009j -0.12868064+0.14559302j -0.00320437+0.0218659j
 -0.02413609+0.00909207j -0.14320363+0.08908738j -0.20218294+0.32073816j
 -0.07670666+0.19701104j -0.05577494+0.20978488j -0.0331994 -0.27551666j
 -0.1274438 -0.07845582j -0.08084521-0.01772735j -0.00917635+0.09198909j
  0.10591547+0.06671533j -0.13166663+0.04434328j -0.08506804+0.10507175j
 -0.1567369 -0.0046447j   0.07730327-0.20998837j -0.01156574-0.08988748j
 -0.00917635-0.00042186j  0.01131862+0.01072202j  0.02728194-0.07789987j
 -0.10257702+0.01991325j -0.10018763+0.10937887j -0.12068261+0.09823499j
 -0.14002801+0.26004461j -0.26128145+0.0361854j  -0.13465263+0.07546983j
 -0.14857901-0.13425951j -0.00590258-0.32448359j -0.09930325-0.12888413j
  0.02732557-0.0895997j   0.04125196+0.12012964j]


In [13]:
dev5_513 = qml.device("default.qubit", wires=5)
print(qml.draw(encoding_map_513)(dev5_513.wires))


0: ─╭●─╭●─╭●─╭●──H─╭IsingZZ(-1.57)─╭IsingZZ(-1.57)─────────────────────────────────────────────────┤
1: ─╰X─│──│──│───H─│───────────────╰IsingZZ(-1.57)─╭IsingZZ(-1.57)─────────────────────────────────┤
2: ────╰X─│──│───H─│───────────────────────────────╰IsingZZ(-1.57)─╭IsingZZ(-1.57)─────────────────┤
3: ───────╰X─│───H─│───────────────────────────────────────────────╰IsingZZ(-1.57)─╭IsingZZ(-1.57)─┤
4: ──────────╰X──H─╰IsingZZ(-1.57)─────────────────────────────────────────────────╰IsingZZ(-1.57)─┤

   
   
   
   
   


In [112]:
def Variation_circuit_5(wires, alpha):
    for wire in wires:
        qml.RZ(alpha[0], wires=wire)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[3, 4])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=2)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])
    qml.IsingZZ(alpha[1], wires=[3, 4])

    qml.RX(alpha[2], wires=0)
    qml.RX(alpha[2], wires=2)
    qml.RX(alpha[2], wires=4)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=2)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=2)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[3, 4])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])  

    qml.RX(alpha[2], wires=2)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])  

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[3, 4])
    
    qml.RX(alpha[2], wires=0)
    qml.RX(alpha[2], wires=4)

    for wire in wires:
        qml.RZ(alpha[0], wires=wire)

In [ ]:
dev5_VGQEC = qml.device("default.qubit", wires=5)
def VGQEC_5_encoding(wires, alpha):
    encoding_map_repetition(wires)
    qml.IsingZZ(-np.pi/2, wires=[4, 0])
    qml.IsingZZ(-np.pi/2, wires=[0, 1])
    qml.IsingZZ(-np.pi/2, wires=[1, 2])
    qml.IsingZZ(-np.pi/2, wires=[2, 3])
    qml.IsingZZ(-np.pi/2, wires=[3, 4])
    Variation_circuit_5(wires, alpha)

In [107]:
print(qml.draw(VGQEC_5_encoding)(dev5_VGQEC.wires, [np.pi, np.pi, np.pi]))

0: ──H─╭●─╭●─╭●─╭●──H─╭IsingZZ(-1.57)─╭IsingZZ(-1.57)──RZ(3.14)──────────────────────
1: ────╰X─│──│──│───H─│───────────────╰IsingZZ(-1.57)─╭IsingZZ(-1.57)──RZ(3.14)──────
2: ───────╰X─│──│───H─│───────────────────────────────╰IsingZZ(-1.57)─╭IsingZZ(-1.57)
3: ──────────╰X─│───H─│───────────────────────────────────────────────╰IsingZZ(-1.57)
4: ─────────────╰X──H─╰IsingZZ(-1.57)────────────────────────────────────────────────

──╭IsingZZ(3.14)────────────────────────────────────────────────────────────────────────────
──╰IsingZZ(3.14)───RX(3.14)─╭IsingZZ(3.14)────────────────────────────────────╭IsingZZ(3.14)
───RZ(3.14)─────────────────╰IsingZZ(3.14)───────────╭IsingZZ(3.14)──RX(3.14)─╰IsingZZ(3.14)
──╭IsingZZ(-1.57)──RZ(3.14)─╭IsingZZ(3.14)──RX(3.14)─╰IsingZZ(3.14)─────────────────────────
──╰IsingZZ(-1.57)──RZ(3.14)─╰IsingZZ(3.14)──────────────────────────────────────────────────

─────────────────╭IsingZZ(3.14)──RX(3.14)──────────────────────────────────────────────────
───RX(3.14)

In [80]:
def repetition_code_recovery_3(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 1])
    qml.Toffoli(wires=[2, 1, 0])


In [ ]:
dev3_rep_circuit = qml.device("default.mixed", wires=3)
@qml.qnode(dev3_rep_circuit)
def circuit_3(wires, alpha):
    #encoding
    encoding_map_repetition(wires)
    #noise
    qml.AmplitudeDamping(0.1, wires=0)
    #decoding
    repetition_code_recovery_3(wires)
    return qml.density_matrix(wires=0)
    
print(qml.draw(circuit_3)(dev3_rep_circuit.wires, 0.9))


0: ─╭●─╭●──H──AmplitudeDamping(0.10)──H─╭●─╭●─╭X─┤  State
1: ─╰X─│───H──H─────────────────────────│──╰X─├●─┤       
2: ────╰X──H──H─────────────────────────╰X────╰●─┤       


In [148]:
print(circuit_3(dev3_rep_circuit.wires, 0.1))

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [85]:
def repetition_code_recovery_5(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    qml.CNOT(wires=[0, 4])
    qml.CNOT(wires=[0, 3])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 1])

    qml.MultiControlledX(control_wires=[1, 2, 3], wires=0)
    qml.MultiControlledX(control_wires=[4, 3, 1], wires=0)
    qml.MultiControlledX(control_wires=[4, 3, 2], wires=0)
    qml.MultiControlledX(control_wires=[4, 2, 1], wires=0)
    qml.MultiControlledX(control_wires=[4, 3, 2, 1], wires=0)

In [90]:
dev5_rep_circuit = qml.device("default.mixed", wires=5)
@qml.qnode(dev5_rep_circuit)
def circuit_5(wires, alpha):
    encoding_map_repetition(wires)
    # qml.AmplitudeDamping(alpha, wires=[0])
    #noise
    qml.PauliZ(wires=0)
    #recovery
    repetition_code_recovery_5(wires)
    
    return qml.density_matrix(wires=0)

print(qml.draw(circuit_5)(dev5_rep_circuit.wires, 0.1))


0: ─╭●─╭●─╭●─╭●──H──Z──H─╭●─╭●─╭●─╭●─╭X─╭X─╭X─╭X─╭X─┤  State
1: ─╰X─│──│──│───H──H────│──│──│──╰X─├●─├●─│──├●─├●─┤       
2: ────╰X─│──│───H──H────│──│──╰X────├●─│──├●─├●─├●─┤       
3: ───────╰X─│───H──H────│──╰X───────╰●─├●─├●─│──├●─┤       
4: ──────────╰X──H──H────╰X─────────────╰●─╰●─╰●─╰●─┤       


In [91]:
print(circuit_5(dev5_rep_circuit.wires, 0.1))

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [20]:
# dev_var = qml.device("default.qubit", wires=5)
# def encoding_variational(params):
#     for wire in dev_var.wires:
#         qml.RZ(params[0], wires=wire)

#     qml.IsingZZ(params[1], wires=[0, 1])
#     qml.IsingZZ(params[1], wires=[3, 4])

#     qml.RX(params[2], wires=1)
#     qml.RX(params[2], wires=3)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])

#     qml.RX(params[2], wires=2)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])

#     qml.RX(params[2], wires=1)
#     qml.RX(params[2], wires=3)

#     qml.IsingZZ(params[1], wires=[0, 1])
#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])
#     qml.IsingZZ(params[1], wires=[3, 4])

#     qml.RX(params[2], wires=0)
#     qml.RX(params[2], wires=2)
#     qml.RX(params[2], wires=4)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])

#     qml.RX(params[2], wires=1)
#     qml.RX(params[2], wires=2)
#     qml.RX(params[2], wires=3)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])

#     qml.RX(params[2], wires=2)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])

#     qml.RX(params[2], wires=1)
#     qml.RX(params[2], wires=3)

#     qml.IsingZZ(params[1], wires=[0, 1])
#     qml.IsingZZ(params[1], wires=[3, 4])

#     qml.RX(params[2], wires=1)
#     qml.RX(params[2], wires=3)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])  

#     qml.RX(params[2], wires=2)

#     qml.IsingZZ(params[1], wires=[1, 2])
#     qml.IsingZZ(params[1], wires=[2, 3])  

#     qml.RX(params[2], wires=1)
#     qml.RX(params[2], wires=3)

#     qml.IsingZZ(params[1], wires=[0, 1])
#     qml.IsingZZ(params[1], wires=[3, 4])
    
#     qml.RX(params[2], wires=0)
#     qml.RX(params[2], wires=4)

#     for wire in dev_var.wires:
#         qml.RZ(params[0], wires=wire)



In [27]:
# print(qml.draw(encoding_variational)(np.array([0.1, 0.2, 0.3])))